In [ ]:
import matplotlib
matplotlib.use('TkAgg')
from pylab import *
import networkx as nx
import random as rd

def initialize():
    global g, nextg
    g = nx.karate_club_graph()
    for i, j in g.edges:
        g.edges[i,j]['weight'] = rd.uniform(0.05,1)
    for i in g.nodes:
        g.nodes[i]['opinion'] = rd.uniform(0,1)
    nextg = g.copy()
    nextg.pos = g.pos = nx.spring_layout(g)

def observe():
    global g, nextg
    cla()
    nx.draw(g, vmin = 0, vmax = 1,
            node_color = [g.nodes[i]['opinion'] for i in g.nodes],
            edge_cmap = cm.binary, edge_vmin = 0, edge_vmax = 1,
            edge_color = [g.edges[i,j]['weight'] for i, j in g.edges],
            pos = g.pos)

alpha = 0.03 # diffusion constant
beta = 0.3 # rate of adaptive edge weight change
gamma = 4 # pickiness of nodes

def update():
    global g, nextg
    connection = rd.choice(list(g.edges))
    delta_o_i = alpha*g.edges[connection]['weight']*(g.nodes[connection[1]]['opinion']-g.nodes[connection[0]]['opinion'])
    g.nodes[connection[0]]['opinion'] += delta_o_i
    delta_o_j = alpha*g.edges[connection]['weight']*(g.nodes[connection[0]]['opinion']-g.nodes[connection[1]]['opinion'])
    g.nodes[connection[1]]['opinion'] += delta_o_j
    
    delta_w = beta*g.edges[connection]['weight']*(1-g.edges[connection]['weight'])*(1-gamma*abs(g.nodes[connection[0]]['opinion']-g.nodes[connection[1]]['opinion']))
    g.edges[connection]['weight'] += delta_w
    
    if g.edges[connection]['weight'] <= 0.05:
        g.remove_edge(connection[0], connection[1])
        
    if rd.uniform(0,1) < 0.01:
        g.add_edges_from([rd.choice(list(g.edges)), rd.choice(list(g.edges))], weight=0.5)
    
    nextg.pos = nx.spring_layout(nextg, pos = g.pos, iterations = 5)
    g, nextg = nextg, g

import pycxsimulator
pycxsimulator.GUI().start(func=[initialize, observe, update])